# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

E. Schinnerer  ->  E. Schinnerer  |  ['E. Schinnerer']
J. Neumann  ->  J. Neumann  |  ['J. Neumann']
N. Neumayer  ->  N. Neumayer  |  ['N. Neumayer']
F. Pinna  ->  F. Pinna  |  ['F. Pinna']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
M. Güdel  ->  M. Güdel  |  ['M. Güdel']


Smith  ->  M. Smith  |  ['Smith']
Zhang  ->  X. Zhang  |  ['Zhang']
L. Boogaard  ->  L. Boogaard  |  ['L. Boogaard']
Arxiv has 52 new papers today
          5 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/4 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2305.14437


extracting tarball to tmp_2305.14437...

 done.


/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3508: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3508: LatexWarning: Found documentclass in tmp_2305.14437/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:399: LatexWarning: Latex injecting: 'authors' from 'tmp_2305.14437/authors.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


...

 done.
Retrieving document from  https://arxiv.org/e-print/2305.14971


extracting tarball to tmp_2305.14971...

 done.
Retrieving document from  https://arxiv.org/e-print/2305.15308


extracting tarball to tmp_2305.15308...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2305.14437-b31b1b.svg)](https://arxiv.org/abs/arXiv:2305.14437) | **Fuelling the nuclear ring of NGC 1097**  |
|| M. C. Sormani, et al. -- incl., <mark>E. Schinnerer</mark>, <mark>J. Neumann</mark>, <mark>N. Neumayer</mark>, <mark>F. Pinna</mark> |
|*Appeared on*| *2023-05-25*|
|*Comments*| *Accepted in MNRAS*|
|**Abstract**| Galactic bars can drive cold gas inflows towards the centres of galaxies. The gas transport happens primarily through the so-called bar ``dust lanes'', which connect the galactic disc at kpc scales to the nuclear rings at hundreds of pc scales much like two gigantic galactic rivers. Once in the ring, the gas can fuel star formation activity, galactic outflows, and central supermassive black holes. Measuring the mass inflow rates is therefore important to understanding the mass/energy budget and evolution of galactic nuclei. In this work, we use CO datacubes from the PHANGS-ALMA survey and a simple geometrical method to measure the bar-driven mass inflow rate onto the nuclear ring of the barred galaxy NGC~1097. The method assumes that the gas velocity in the bar lanes is parallel to the lanes in the frame co-rotating with the bar, and allows one to derive the inflow rates from sufficiently sensitive and resolved position-position-velocity diagrams if the bar pattern speed and galaxy orientations are known. We find an inflow rate of $\dot{M}=(3.0 \pm 2.1)\, \rm M_\odot\, yr^{-1}$ averaged over a time span of 40 Myr, which varies by a factor of a few over timescales of $\sim$10 Myr. Most of the inflow appears to be consumed by star formation in the ring which is currently occurring at a rate of ${\rm SFR}\simeq~1.8$-$2 \rm M_\odot\, yr^{-1}$, suggesting that the inflow is causally controlling the star formation rate in the ring as a function of time. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2305.14895-b31b1b.svg)](https://arxiv.org/abs/arXiv:2305.14895) | **The Lobster Eye Imager for Astronomy Onboard the SATech-01 Satellite**  |
|| Z. Ling, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2023-05-25*|
|*Comments*| *Accepted by RAA*|
|**Abstract**| The Lobster Eye Imager for Astronomy (LEIA), a pathfinder of the Wide-field X-ray Telescope of the Einstein Probe (EP) mission, was successfully launched onboard the SATech-01 satellite of the Chinese Academy of Sciences on 27 July 2022. In this paper, we introduce the design and on-ground test results of the LEIA instrument. Using state-of-the-art Micro-Pore Optics (MPO), a wide field-of-view (FoV) of 346 square degrees (18.6 degrees * 18.6 degrees) of the X-ray imager is realized. An optical assembly composed of 36 MPO chips is used to focus incident X-ray photons, and four large-format complementary metal-oxide semiconductor (CMOS) sensors, each of 6 cm * 6 cm, are used as the focal plane detectors. The instrument has an angular resolution of 4 - 8 arcmin (in FWHM) for the central focal spot of the point spread function, and an effective area of 2 - 3 cm2 at 1 keV in essentially all the directions within the field of view. The detection passband is 0.5 - 4 keV in the soft X-rays and the sensitivity is 2 - 3 * 10-11 erg s-1 cm-2 (about 1 mini-Crab) at 1,000 second observation. The total weight of LEIA is 56 kg and the power is 85 W. The satellite, with a design lifetime of 2 years, operates in a Sun-synchronous orbit of 500 km with an orbital period of 95 minutes. LEIA is paving the way for future missions by verifying in flight the technologies of both novel focusing imaging optics and CMOS sensors for X-ray observation, and by optimizing the working setups of the instrumental parameters. In addition, LEIA is able to carry out scientific observations to find new transients and to monitor known sources in the soft X-ray band, albeit limited useful observing time available. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2305.14971-b31b1b.svg)](https://arxiv.org/abs/arXiv:2305.14971) | **Complete X-ray census of Mdwarfs in the solar Neighborhood I. GJ 745 AB:  Coronal-hole Stars in the 10 pc Sample**  |
|| M. Caramazza, et al. -- incl., <mark>M. Güdel</mark> |
|*Appeared on*| *2023-05-25*|
|*Comments*| *accepted for publication in Astronomy & Astrophysics (A&A)*|
|**Abstract**| We have embarked in a systematic study of the X-ray emission in a volume-limited sample of M dwarf stars, in order to explore the full range of activity levels present in their coronae and, thus, to understand the conditions in their outer atmospheres and their possible impact on the circumstellar environment. We identify in a recent catalog of the Gaia objects within 10 pc from the Sun all the stars with spectral type between M0 and M4, and search systematically for X-ray measurements of this sample. To this end, we use both archival data (from ROSAT, XMM-Newton, and from the ROentgen Survey with an Imaging Telescope Array (eROSITA) onboard the Russian Spektrum-Roentgen-Gamma mission) and our own dedicated XMM-Newton observations. To make inferences on the properties of the M dwarf corona we compare the range of their observed X-ray emission levels to the flux radiated by the Sun from different types of magnetic structures: coronal holes, background corona, active regions and cores of active regions. At the current state of our project, with more than 90\% of the 10pc M dwarf sample observed in X-rays, only GJ 745 A has no detection. With an upper limit luminosity of log Lx [erg/s] < 25.4 and an X-ray surface flux of log FX,SURF [erg/cm^2/s] < 3.6 GJ 745 A defines the lower boundary of the X-ray emission level of M dwarfs. Together with its companion GJ 745 B, GJ 745 A it is the only star in this volume-complete sample located in the range of FX,SURF that corresponds to the faintest solar coronal structures, the coronal holes. The ultra-low X-ray emission level of GJ 745 B (log Lx [erg/s] = 25.6 and log FX,SURF [erg/cm^2/s] = 3.8) is entirely attributed to flaring activity, indicating that, while its corona is dominated by coronal holes, at least one magnetically active structure is present and determines the total X-ray brightness and the coronal temperature of the star. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2305.15308-b31b1b.svg)](https://arxiv.org/abs/arXiv:2305.15308) | **Beyond the disk: EUV coronagraphic observations of the Extreme  Ultraviolet Imager on board Solar Orbiter**  |
|| Auchère, et al. -- incl., <mark>Smith</mark>, <mark>Zhang</mark> |
|*Appeared on*| *2023-05-25*|
|*Comments*| **|
|**Abstract**| Most observations of the solar corona beyond 2 Rs consist of broadband visible light imagery from coronagraphs. The associated diagnostics mainly consist of kinematics and derivations of the electron number density. While the measurement of the properties of emission lines can provide crucial additional diagnostics of the coronal plasma (temperatures, velocities, abundances, etc.), these observations are comparatively rare. In visible wavelengths, observations at these heights are limited to total eclipses. In the VUV range, very few additional observations have been achieved since the pioneering results of UVCS. One of the objectives of the Full Sun Imager (FSI) channel of the EUI telescope on board the Solar Orbiter mission has been to provide very wide field-of-view EUV diagnostics of the morphology and dynamics of the solar atmosphere in temperature regimes that are typical of the lower transition region and of the corona. FSI carries out observations in two narrowbands of the EUV spectrum centered on 17.4 nm and 30.4 nm that are dominated, respectively, by lines of Fe IX/X (formed in the corona around 1 MK) and by the resonance line of He II (formed around 80 kK in the lower transition region). Unlike previous EUV imagers, FSI includes a moveable occulting disk that can be inserted in the optical path to reduce the amount of instrumental stray light to a minimum. FSI detects signals at 17.4 nm up to the edge of its FOV (7~Rs), which is about twice further than was previously possible. Comparisons with observations by the LASCO and Metis coronagraphs confirm the presence of morphological similarities and differences between the broadband visible light and EUV emissions, as documented on the basis of prior eclipse and space-based observations. The very-wide-field observations of FSI are paving the way for future dedicated instruments. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2305.14437.md
    + _build/html/tmp_2305.14437/./images/inflow_ngc1097_paper.png
    + _build/html/tmp_2305.14437/./images/inflow_ngc1097_spl_07.png
    + _build/html/tmp_2305.14437/./images/rgb_anno_v3.png


## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\di}{\mathrm{d}}$
$\newcommand{\bfx}{\mathbf{x}}$
$\newcommand{\bfe}{\mathbf{e}}$
$\newcommand{\vlos}{{v}_{\rm los}}$
$\newcommand{\NSD}{\mathrm{NSD}}$
$\newcommand{\BAR}{\mathrm{BAR}}$
$\newcommand{\Tspin}{T_{\rm s}}$
$\newcommand{\Tb}{T_{\rm b}}$
$\newcommand{◦ee}{\ensuremath{^\circ}}$
$\newcommand{\Th}{T_{\rm h}}$
$\newcommand{\Tc}{T_{\rm c}}$
$\newcommand{\bfr}{\mathbf{r}}$
$\newcommand{\bfv}{\mathbf{v}}$
$\newcommand{\bfJ}{\mathbf{J}}$
$\newcommand{\pc}{ {\rm pc}}$
$\newcommand{\kpc}{ {\rm kpc}}$
$\newcommand{\Myr}{ {\rm Myr}}$
$\newcommand{\Gyr}{ {\rm Gyr}}$
$\newcommand{\kms}{ {\rm km  s^{-1}}}$
$\newcommand{\de}[2]{\frac{\partial #1}{\partial{#2}}}$
$\newcommand{\cs}{c_{\rm s}}$
$\newcommand{\rb}{r_{\rm b}}$
$\newcommand{\rqu}{r_{\rm q}}$
$\newcommand{\nuP}{\nu_{\rm P}}$
$\newcommand{\thetaobs}{\theta_{\rm obs}}$
$\newcommand{\hatn}{\hat{\textbf{n}}}$
$\newcommand{\hats}{\hat{\textbf{s}}}$
$\newcommand{\hatx}{\hat{\textbf{x}}}$
$\newcommand{\haty}{\hat{\textbf{y}}}$
$\newcommand{\hatz}{\hat{\textbf{z}}}$
$\newcommand{\hatX}{\hat{\textbf{X}}}$
$\newcommand{\hatY}{\hat{\textbf{Y}}}$
$\newcommand{\hatZ}{\hat{\textbf{Z}}}$
$\newcommand{\hatN}{\hat{\textbf{N}}}$
$\newcommand{\pa}{\partial}$
$\newcommand{\e}{\mathrm{e}}$
$\newcommand{\Msun}{  \rm M_\odot}$
$\newcommand{\Msunyr}{  \rm M_\odot  yr^{-1}}$
$\newcommand{\masyr}{  \rm mas  yr^{-1}}$
$\newcommand{\Omegap}{\Omega_{\rm p}}$
$\newcommand{\mcs}[1]{{\textcolor{myblue}{#1}}}$
$\newcommand{\aifa}{Argelander-Institut für Astronomie, Universität Bonn, Auf dem Hügel 71, 53121, Bonn, Germany}$
$\newcommand{\ita}{Universität Heidelberg, Zentrum für Astronomie, Institut für theoretische Astrophysik, Albert-Ueberle-Str. 2, 69120 Heidelberg, Germany}$
$\newcommand{\eso}{European Southern Observatory, Karl-Schwarzschild-Stra{\ss}e 2, 85748 Garching, Germany}$
$\newcommand{\mcmaster}{Department of Physics and Astronomy, McMaster University, 1280 Main Street West, Hamilton, ON L8S 4M1, Canada}$
$\newcommand{\cita}{Canadian Institute for Theoretical Astrophysics (CITA), University of Toronto, 60 St George Street, Toronto, ON M5S 3H8, Canada}$
$\newcommand{\lyon}{Univ Lyon, Univ Lyon1, ENS de Lyon, CNRS, Centre de Recherche Astrophysique de Lyon UMR5574, F-69230 Saint-Genis-Laval France}$
$\newcommand{\OSU}{Department of Astronomy, The Ohio State University, 140 West 18th Avenue, Columbus, Ohio 43210, USA}$
$\newcommand{ÇAPP}{Center for Cosmology and Astroparticle Physics, 191 West Woodruff Avenue, Columbus, OH 43210, USA}$
$\newcommand{\ljmu}{Astrophysics Research Institute, Liverpool John Moores University, 146 Brownlow Hill, Liverpool L3 5RF, UK}$
$\newcommand{\mpia}{Max-Planck-Institut für Astronomie, Königstuhl 17, D-69117 Heidelberg, Germany}$
$\newcommand{\ugent}{Sterrenkundig Observatorium, Universiteit Gent, Krijgslaan 281 S9,$
$B-9000 Gent, Belgium}$
$\newcommand{\ign}{Observatorio Astron{ó}mico Nacional (IGN), C/Alfonso XII, 3, E-$
$28014 Madrid, Spain}$
$\newcommand{\oxford}{Sub-department of Astrophysics, Department of Physics, University of Oxford, Keble Road, Oxford OX1 3RH, UK}$
$\newcommand{\durham}{Institute for Computational Cosmology, Department of Physics, Durham University, South Road, Durham, DH1 3LE, UK}$
$\newcommand{\aapf}{NSF Astronomy and Astrophysics Postdoctoral Fellow}$
$\newcommand{\arizona}{Steward Observatory, University of Arizona, Tucson, AZ 85721, USA}$
$\newcommand{\anu}{Research School of Astronomy and Astrophysics, Australian National University, Canberra, ACT 2611, Australia}$
$\newcommand{\arc}{ARC Centre of Excellence for All Sky Astrophysics in 3 Dimensions (ASTRO 3D), Australia}$
$\newcommand{\iwr}{Universität Heidelberg, Interdisziplinäres Zentrum für Wissenschaftliches Rechnen, Im Neuenheimer Feld 205, D-69120 Heidelberg, Germany}$
$\newcommand{\epfl}{Institute of Physics, Laboratory for galaxy evolution and spectral modelling, EPFL, Observatoire de Sauverny, Chemin Pegais 51, 1290 Versoix, Switzerland}$
$\newcommand{\ualberta}{Dept. of Physics, 4-183 CCIS, University of Alberta, Edmonton, Alberta T6G 2E1, Canada}$
$\newcommand{\tum}{Technical University of Munich, School of Engineering and Design, Department of Aerospace and Geodesy, Chair of Remote Sensing Technology, \\\hspace{2.2mm}Arcisstr. 21, 80333 Munich, Germany}$
$\newcommand{\cool}{Cosmic Origins Of Life (COOL) Research DAO, coolresearch.io}$
$\newcommand{\manch}{Jodrell Bank Centre for Astrophysics, Department of Physics and Astronomy, University of Manchester, Oxford Road, Manchester M13 9PL, UK}$
$\newcommand{\UCSD}{Center for Astrophysics and Space Sciences, Department of Physics, University of California San Diego, 9500 Gilman Drive, La Jolla, CA 92093, USA}$
$\newcommand{\ari}{Astronomisches Rechen-Institut, Zentrum f{ü}r Astronomie der Universit{ä}t Heidelberg, M{ö}nchhofstra{\ss}e 12-14, 69120 Heidelberg,Germany}$
$\newcommand{\CO}[2]{\mbox{\mathrm{CO} (#1\text{--}#2)}}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>



<div id="title">

# Fuelling the nuclear ring of NGC 1097

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2305.14437-b31b1b.svg)](https://arxiv.org/abs/2305.14437)<mark>Appeared on: 2023-05-25</mark> -  _Accepted in MNRAS_

</div>
<div id="authors">

M. C. Sormani, et al. -- incl., <mark>E. Schinnerer</mark>, <mark>J. Neumann</mark>, <mark>N. Neumayer</mark>, <mark>F. Pinna</mark>

</div>
<div id="abstract">

**Abstract:** Galactic bars can drive cold gas inflows towards the centres of galaxies. The gas transport happens primarily through the so-called bar "dust lanes", which connect the galactic disc at kpc scales to the nuclear rings at hundreds of pc scales much like two gigantic galactic rivers. Once in the ring, the gas can fuel star formation activity, galactic outflows, and central supermassive black holes. Measuring the mass inflow rates is therefore important to understanding the mass/energy budget and evolution of galactic nuclei.  In this work, we use CO datacubes from the PHANGS-ALMA survey and a simple geometrical method to measure the bar-driven mass inflow rate onto the nuclear ring of the barred galaxy NGC 1097. The method assumes that the gas velocity in the bar lanes is parallel to the lanes in the frame co-rotating with the bar, and allows one to derive the inflow rates from sufficiently sensitive and resolved position-position-velocity diagrams if the bar pattern speed and galaxy orientations are known. We find an inflow rate of $\dot{M}=(3.0 \pm 2.1)\Msunyr$ averaged over a time span of 40 Myr, which varies by a factor of a few over timescales of $\sim$ 10 Myr. Most of the inflow appears to be consumed by star formation in the ring which is currently occurring at a rate of ${\rm SFR}\simeq 1.8 \mhyphen 2 \Msunyr$ , suggesting that the inflow is causally controlling the star formation rate in the ring as a function of time.

</div>

<div id="div_fig1">

<img src="tmp_2305.14437/./images/inflow_ngc1097_paper.png" alt="Fig7" width="100%"/>

**Figure 7. -** Position-position (top) and position-velocity (bottom) projection of the CO $J=2-1$ datacube of NGC 1097. The galaxy is rotated in the plane of the sky such that the direction of the kinematic position angle points towards the negative $x$ axis (see black dashed line in the top panels). Red and blue contours indicate the emission in the datacube associated with the bar bar lanes. The red and blue lines indicate the spline fits to the bar lanes. The black dashed line in the bottom-right panel indicates the quantity $v_{\rm los}=\Omega_{\rm p} x \sin i$ which appears in the deprojection of the velocity parallel to the bar lanes (see Equation \ref{eq:vpar}). (*fig:dl*)

</div>
<div id="div_fig2">

<img src="tmp_2305.14437/./images/inflow_ngc1097_spl_07.png" alt="Fig5" width="100%"/>

**Figure 5. -** *Top*: the instantaneous mass inflow rate along the bar lanes of NGC 1097 calculated using Eq. $\e$qref{eq:Mdot}. The blue and red dashed lines show the inflow rates averaged over the non-hatched region. *Bottom*: the cumulative mass accreted over time. The hatched area shows where the inflow rate is not reliable due to interaction between the bar lanes and the nuclear ring. (*fig:Mdot*)

</div>
<div id="div_fig3">

<img src="tmp_2305.14437/./images/rgb_anno_v3.png" alt="Fig1" width="100%"/>

**Figure 1. -** Position of bar dust lanes in the nearby, barred spiral galaxy NGC 1097. We show in the background a (WFC3) _ HST_ image composed of the following filters from projects PID 13413 and 15654 (processed as part of PHANGS-HST; see \citealp{Lee2021}): F275W nm (UV) and F336W (u) broad-band in violet, F438W (B) broad-band in dark blue, F547M (Strömgren y) medium-band in cyan, F555W broad-band (V) in green, F814W broad-band (I) in yellow, and F657N (H$\alpha$ + [N II]) in red. Overlaid as coloured contours is the CO(2-1) integrated intensity from the PHANGS-ALMA survey \citep{Leroy2021survey}, in levels of 2, 10, 20, 50, 75, 100, 250 K km s$^{-1}$(increasing from blue to red). Note that the galaxy is rotated in the plane of the sky such that the direction of the kinematic position angle points towards the negative $x$ axis (see Fig. \ref{fig:dl}). (*fig:rgb*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2305.14437"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

210  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

3  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

1  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
